In [5]:
pip install langchain-community faiss-cpu openai tiktoken

Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores.faiss import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
import faiss

In [14]:
api_key=''

In [9]:
# Define user-specific events
user_data = {
    "user_1": [
  {"event_id": "1", "subject": "User 1 Event 1", "start_time": "2024-11-10 11:00", "description": "Description for User 1 Event 1"},
  {"event_id": "2", "subject": "User 1 Event 2", "start_time": "2024-11-11 13:00", "description": "Description for User 1 Event 2"},
  {"event_id": "3", "subject": "User 1 Event 3", "start_time": "2024-11-11 09:00", "description": "Description for User 1 Event 3"},
  {"event_id": "4", "subject": "User 1 Event 4", "start_time": "2024-11-12 11:00", "description": "Description for User 1 Event 4"},
  {"event_id": "5", "subject": "User 1 Event 5", "start_time": "2024-11-12 13:00", "description": "Description for User 1 Event 5"},
  {"event_id": "6", "subject": "User 1 Event 6", "start_time": "2024-11-13 09:00", "description": "Description for User 1 Event 6"},
  {"event_id": "7", "subject": "User 1 Event 7", "start_time": "2024-11-13 11:00", "description": "Description for User 1 Event 7"},
  {"event_id": "8", "subject": "User 1 Event 8", "start_time": "2024-11-14 13:00", "description": "Description for User 1 Event 8"},
  {"event_id": "9", "subject": "User 1 Event 9", "start_time": "2024-11-14 09:00", "description": "Description for User 1 Event 9"},
  {"event_id": "10", "subject": "User 1 Event 10", "start_time": "2024-11-15 11:00", "description": "Description for User 1 Event 10"},
  {"event_id": "30", "subject": "User 1 Event 30", "start_time": "2024-11-25 09:00", "description": "Description for User 1 Event 30"}
],

    "user_2": [
  {"event_id": "1", "subject": "User 2 Event 1", "start_time": "2024-11-10 11:00", "description": "Description for User 2 Event 1"},
  {"event_id": "2", "subject": "User 2 Event 2", "start_time": "2024-11-11 13:00", "description": "Description for User 2 Event 2"},
  {"event_id": "3", "subject": "User 2 Event 3", "start_time": "2024-11-11 09:00", "description": "Description for User 2 Event 3"},
  {"event_id": "4", "subject": "User 2 Event 4", "start_time": "2024-11-12 11:00", "description": "Description for User 2 Event 4"},
  {"event_id": "5", "subject": "User 2 Event 5", "start_time": "2024-11-12 13:00", "description": "Description for User 2 Event 5"},
  {"event_id": "6", "subject": "User 2 Event 6", "start_time": "2024-11-13 09:00", "description": "Description for User 2 Event 6"},
  {"event_id": "7", "subject": "User 2 Event 7", "start_time": "2024-11-13 11:00", "description": "Description for User 2 Event 7"},
  {"event_id": "8", "subject": "User 2 Event 8", "start_time": "2024-11-14 13:00", "description": "Description for User 2 Event 8"},
  {"event_id": "9", "subject": "User 2 Event 9", "start_time": "2024-11-14 09:00", "description": "Description for User 2 Event 9"},
  {"event_id": "10", "subject": "User 2 Event 10", "start_time": "2024-11-15 11:00", "description": "Description for User 2 Event 10"},
  {"event_id": "30", "subject": "User 2 Event 30", "start_time": "2024-11-25 09:00", "description": "Description for User 2 Event 30"}
]
}


In [10]:
# Function to create FAISS index and memory for a specific user
def setup_user_data(user_id, events):
    # Prepare documents
    documents = [
        Document(page_content=f"{event['subject']} on {event['start_time']} - {event['description']}", metadata=event)
        for event in events
    ]

    # Generate embeddings
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", api_key=api_key)
    embeddings = embedding_model.embed_documents([doc.page_content for doc in documents])
    
    # Create FAISS index
    dimension = len(embeddings[0])
    index = faiss.IndexFlatL2(dimension)
    docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(documents)})
    index_to_docstore_id = {i: str(i) for i in range(len(documents))}
    
    faiss_index = FAISS(
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id,
        embedding_function=embedding_model.embed_query
    )
    faiss_index.add_texts([doc.page_content for doc in documents])
    
    # Create memory for the user
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    return faiss_index, memory

In [17]:
# Setup data for both users
user_chains = {}
for user_id, events in user_data.items():
    faiss_index, memory = setup_user_data(user_id, events)
    retriever = faiss_index.as_retriever(search_kwargs={"k": 3})
    prompt_template = PromptTemplate.from_template(template="""
        You are a helpful assistant providing information about events.

        Context: {context}

        Chat History: {chat_history}

        Question: {question}

        Answer:
    """)
    llm = ChatOpenAI(temperature=0.3, model="gpt-4o-mini", openai_api_key=api_key)
    
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        return_source_documents=False,
        combine_docs_chain_kwargs={"prompt": prompt_template}
    )
    user_chains[user_id] = qa_chain

/tmp/ipykernel_30/1558425398.py:28: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/tmp/ipykernel_30/1703695599.py:17: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.3, model="gpt-4o-mini", openai_api_key=api_key)


In [19]:
# Example usage for both users
def query_user(user_id, question):
    qa_chain = user_chains[user_id]
    response = qa_chain.invoke({"question": question, "chat_history": []})
    return response["answer"]

# Test queries
print("User 1 Query:")
print(query_user("user_1", "What events do I have this week?"))


User 1 Query:
You have the following events this week:

1. **User 1 Event 1** on **November 10, 2024** at **11:00** - Description for User 1 Event 1
2. **User 1 Event 7** on **November 13, 2024** at **11:00** - Description for User 1 Event 7
3. **User 1 Event 8** on **November 14, 2024** at **13:00** - Description for User 1 Event 8

If you need more details about any of these events, feel free to ask!


In [20]:
print("User 2 Query:")
print(query_user("user_2", "Can I go golfing tomorrow"))


User 2 Query:
Based on your event schedule for tomorrow, November 11, 2024, you have the following events:

- **User 2 Event 3** at 09:00 - Description for User 2 Event 3
- **User 2 Event 4** at 11:00 - Description for User 2 Event 4

Since your events start at 09:00 and end around 11:00, you may have some time available for golfing after your events. However, please consider the duration of your events and any travel time needed to get to the golf course. If you finish your events on time and have enough daylight, you should be able to go golfing.
